## BERT для вопросно-ответных систем

Научим BERT отвечать на вопросы (question-answering). Стандартный датасет для тренировки QA моделей (или question-answering моделей) называется SQUAD[2] и расшифровывается как Stanford Question Answering Dataset. Он включает в себя вопросы и ответы на них по достаточно разнообразной тематике. Вопросы формировались на основе статей или отрывков из статьи из Википедии. Ответ на каждый вопрос представляет собой сегмент текста или промежуток из соответствующего отрывка. В новой версии датасета (в 2.0 версии) даже возможны вопросы без ответа. Имеется в виду, что есть вопросы, для ответа на которые недостаточно информации в предложенном фрагменте текста. Таким образом, алгоритм составления этого датасета был следующим. Асессор читал фрагмент текста из Википедии (как правило, всего несколько абзацев), затем формулировал вопрос по прочитанному и фиксировал правильный ответ. На большинство вопросов присутствует несколько вариантов ответа. Но, как правило, это всего лишь переформулировки одного и того же, по смыслу, варианта ответа. Например, на вопрос "когда началась эпоха Возрождения в Италии" можно ответить "14 век", "в 14 веке", "в начале 14 века" или как-то ещё, использовав слова "14" и "век". Учитывая специфику нашего датасета, на вход нейросети мы будем подавать не только сам вопрос, но и соответствующий фрагмент текста, параграф текста. А в качестве выхода нейросети будем ожидать две позиции в тексте: начало ответа и конец ответа. Два файла, один — с обучающей выборкой (train.json), и с выборкой, на которой мы будем тестировать наш алгоритм (он называется dev_version2.json). 

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

# !git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
# import sys; sys.path.append('./stepik-dl-nlp')

Скачайте датасет (SQuAD) [отсюда](https://rajpurkar.github.io/SQuAD-explorer/). Для выполенения семинара Вам понадобятся файлы `train-v2.0.json` и `dev-v2.0.json`.

Склонируйте репозиторий https://github.com/huggingface/transformers (воспользуйтесь скриптом `clone_pytorch_transformers.sh`) и положите путь до папки `examples` в переменную `PATH_TO_EXAMPLES`. 

Итак, давайте сначала склонируем (clone) репозитории и положим путь до папки "examples" в переменную path_to_examples.  

In [1]:
PATH_TO_TRANSFORMERS_REPO = './datasets/transformers/'
import os
os.environ['PATH_TO_TRANSFORMER_REPO'] = PATH_TO_TRANSFORMERS_REPO
# ! bash clone_pytorch_transformers.sh $PATH_TO_TRANSFORMERS_REPO
import sys

PATH_TO_EXAMPLES = os.path.join(PATH_TO_TRANSFORMERS_REPO, 'examples')
sys.path.append(PATH_TO_EXAMPLES)
import torch
import tqdm
import json


from utils_squad import (read_squad_examples, convert_examples_to_features,
                         RawResult, write_predictions,
                         RawResultExtended, write_predictions_extended)

from run_squad import train, load_and_cache_examples


In [ ]:

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)

from transformers import (WEIGHTS_NAME, BertConfig, XLNetConfig, XLMConfig,
                          BertForQuestionAnswering, BertTokenizer)

from utils_squad_evaluate import EVAL_OPTS, main as evaluate_on_squad

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Дальше, так же как и в прошлом семинаре, загружаем токенизатор для BERT и готовую модификацию BERT для вопросно-ответных систем под названием "bert for question answering" из библиотеки pytorch-transformers. Выполняем вот этот кусочек кода. Загрузка модели может занять какое-то время. 

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Отлично, модель скачалась. Теперь у вас есть два варианта развития событий — простой и чуть-чуть более сложный. Вы можете попробовать дообучить модель на датасете SQUAD[2] самостоятельно или же загрузить уже предобученные веса сети и перейти сразу к блоку "оценка качества работы модели".[1] Подгрузить веса нейросети можно с помощью функции "load_state_dict", как вы уже помните. И при подгрузке весов в случае отсутствия GPU не забудьте указать параметр "map_location" (вот как в этой строке кода). [1] https://stepik.org/lesson/268748/step/7?unit=249768 [2] https://rajpurkar.github.io/SQuAD-explorer/

Мы хотим натренировать сеть отвечать на вопросы по статьям с Википедии. Будем использовать для обучения датасет SQuAD (Stanford Question Answering Dataset). Какую информацию нужно подавать на вход нейросети во время обучения?

-Несколько произвольных параграфов текста, не имеющих отношения к вопросу  

+Правильный ответ на вопрос  

+Вопрос  

+Параграф текста, по которому задается вопрос  

In [ ]:
# если Вы не хотите запускать файн-тюнинг, пропустите блок "Дообучение",
# подгрузите веса уже дообученной модели и переходите к блоку "Оценка качества"

# скачайте веса с Google-диска и положите их в папку models
# https://drive.google.com/drive/folders/1-DR30q7MF-gZ51TDx596dAOhgh-uOAPj?usp=sharing

In [ ]:
if torch.cuda.is_available():
    model.cuda()
    model.load_state_dict(torch.load('models/bert_squad_1epoch.pt')) # если у вас есть GPU
else:
    model.load_state_dict(torch.load('models/bert_squad_1epoch.pt', map_location=device)) # если GPU нет

### Дообучение

Итак, рассмотрим процесс дообучения модели. Аналогичный результат можно получить, просто запустив скрипт run_squad.py[1] из папки "examples". Первый параметр называется "train_file" — сюда мы кладём путь до обучающей выборки, в переменную "predict_file", соответственно, кладём путь до датасета, на котором мы будем тестироваться и оценивать качество работы модели. Ещё из интересных параметров — у нас есть параметр "model_type", здесь нужно указать, с какой моделью мы будем работать (в нашем случае, это BERT, но также можно было указать, например, XLNet на DistilBERT). А в параметр "model_name_or_path" мы кладём либо путь до предобученной модели, если таковая у вас есть, либо же название модели из переменной "all_models". Эту переменную "all_models" я определила после этого класса — вот она, давайте посмотрим, что в ней содержится. В этой переменной есть все варианты моделей, с которыми мы можем работать с помощью данного кода (с помощью кодов библиотеки pytorch-transformers). Если говорить про BERT, то у нас есть достаточно вариантов — есть варианты моделей BERT Base и BERT Large, можно использовать "uncased" или "cased" варианты, можно брать multilingual-модели, либо же какие-то специально заточенные под конкретный язык (например, BERT для китайского языка). Мы же будем использовать самый простой вариант "bert-base-uncased". Давайте продолжим наше путешествие по параметрам, которые мы будем использовать для обучения модели. Ещё из интересных параметров стоит отметить "version_2_with_negative" — этот параметр отвечает за то, будем ли мы при обучении учитывать вопросы, на которые ответа в датасете нет. Также мы можем указать максимальную длину параграфа, который мы подаём в нашу нейросеть (в нашем случае это будет 384, имеется виду 384 токена, а не 384 символа). Также можно указать страйд — параметр "doc_stride" отвечает за то, сколько страйда мы будем использовать при делении длинного документа на чанки (chunks), на небольшие кусочки. Кроме того, можно указать максимальное количество токенов в вопросе — за это отвечает параметр "max_query_length" (в нашем случае, это 128) и, также, можно указать максимальную длину ответа — это параметр "max_answer_length". Кроме того, давайте посмотрим на параметры, которые нам нужны непосредственно для обучения нейросети. А именно, мы указываем "learning_rate", указываем "weight_decay" (в нашем случае, мы его не используем, он равен нулю). Можем указать эпсилон для оптимизатора Adam, также мы, конечно, указываем, сколько эпох мы будем тренировать нашу модель. В нашем случае это всего 5 эпох. Кроме того, мы указываем, сколько у нас будет warmup-шагов (в нашем случае мы вовсе не будем использовать warm-up), ещё из интересных параметров стоит отметить вот эти две вещи — "logging_steps" и "save_steps". Мы говорим, через сколько шагов мы хотим логировать (log) прогресс нашей модели и сохранять чекпойнты (checkpoints). В нашем случае — будем сохранять чекпойнты каждые 5 000 шагов. Также, если у нас выставлен параметр "evaluate_during_training", через каждые 5 000 шагов наша модель будет оценивать качество своей работы на датасете из predict_file (на датасете, который лежит вот по этому пути). И также наша модель будет выводить нам список метрик, которые удалось достичь на текущий момент тренировки. Кроме того, мы можем указать — печатать ли warnings, можем указать, что мы не хотим использовать CUDA (даже если у нас доступна GPU). Кроме того, можем задать размеры батча для тренировки и для evaluation. [1] https://github.com/huggingface/transformers/blob/master/examples/run_squad.py

In [ ]:
# !pip install dataclasses
from dataclasses import dataclass

@dataclass
class TRAIN_OPTS:
    train_file : str = 'train-v2.0.json'    # SQuAD json-файл для обучения
    predict_file : str = 'dev-v2.0.json'    # SQuAD json-файл для тестирования
    model_type : str = 'bert'               # тип модели (может быть  'bert', 'xlnet', 'xlm', 'distilbert')
    model_name_or_path : str = 'bert-base-uncased' # путь до предобученной модели или название модели из ALL_MODELS
    output_dir : str = '/tmp' # путь до директории, где будут храниться чекпоинты и предсказания модели
    device : str = 'cuda' # cuda или cpu
    n_gpu : int = 1 # количество gpu для обучения
    cache_dir : str = '' # где хранить предобученные модели, загруженные с s3
        
    # Если true, то в датасет будут включены вопросы, на которые нет ответов.
    version_2_with_negative : bool = True
    # Если (null_score - best_non_null) больше, чем порог, предсказывать null.
    null_score_diff_threshold : float = 0.0
    # Максимальная длина входной последовательности после WordPiece токенизации. Sequences 
    # Последовательности длиннее будут укорочены, для более коротких последовательностей будет использован паддинг
    max_seq_length : int = 384
    # Сколько stride использовать при делении длинного документа на чанки
    doc_stride : int = 128
    # Максимальное количество токенов в вопросе. Более длинные вопросы будут укорочены до этой длины
    max_query_length : int = 128 #
        
    do_train : bool = True
    do_eval : bool = True
        
    # Запускать ли evaluation на каждом logging_step
    evaluate_during_training : bool = True
    # Должно быть True, если Вы используете uncased модели
    do_lower_case : bool = True #
    
    per_gpu_train_batch_size : int = 8 # размер батча для обучения
    per_gpu_eval_batch_size : int = 8 # размер батча для eval
    learning_rate : float = 5e-5 # learning rate
    gradient_accumulation_steps : int = 1 # количество шагов, которые нужно сделать перед backward/update pass
    weight_decay : float = 0.0 # weight decay
    adam_epsilon : float = 1e-8 # эпсилон для Adam
    max_grad_norm : float = 1.0 # максимальная норма градиента
    num_train_epochs : float = 5.0 # количество эпох на обучение
    max_steps : int = -1 # общее количество шагов на обучение (override num_train_epochs)
    warmup_steps : int = 0 # warmup 
    n_best_size : int = 5 # количество ответов, которые надо сгенерировать для записи в nbest_predictions.json
    max_answer_length : int = 30 # максимально возможная длина ответа
    verbose_logging : bool = True # печатать или нет warnings, относящиеся к обработке данных
    logging_steps : int = 5000 # логировать каждые X шагов
    save_steps : int = 5000 # сохранять чекпоинт каждые X шагов
        
    # Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number
    eval_all_checkpoints : bool = True
    no_cuda : bool = False # не использовать CUDA
    overwrite_output_dir : bool = True # переписывать ли содержимое директории с выходными файлами
    overwrite_cache : bool = True # переписывать ли закешированные данные для обучения и evaluation
    seed : int = 42 # random seed
    local_rank : int = -1 # local rank для распределенного обучения на GPU
    fp16 : bool = False # использовать ли 16-bit (mixed) precision (через NVIDIA apex) вместо 32-bit"
    # Apex AMP optimization level: ['O0', 'O1', 'O2', and 'O3'].
    # Подробнее тут: https://nvidia.github.io/apex/amp.html
    fp16_opt_level : str = '01'

In [ ]:
ALL_MODELS = sum((tuple(conf.pretrained_config_archive_map.keys()) \
                  for conf in (BertConfig, XLNetConfig, XLMConfig)), ())
ALL_MODELS

 мы рассмотрели основные параметры, которые нам нужны для тренировки модели, теперь можем подгрузить наши данные и запустить тренировку. Давайте загрузим наш датасет. Обратите внимание на параметр "evaluate" в этой строчке кода. Если здесь будет написано "evaluate = True", то никакого обучения происходить не будет. Запустится evaluation на наших данных, а веса модели меняться никак не будут. 

In [ ]:
args = TRAIN_OPTS()
train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False)

После того, как мы загрузили наш датасет (это достаточно долгий процесс...), мы можем запустить обучение нашей модели (с помощью вот этой строчки кода). Полное обучение всех пяти эпох, как я уже говорила, заняло около 7 часов, поэтому запускать эту ячейку мы не будем. Отмечу лишь, что через каждые 5 000 шагов, как и обсуждалось ранее, у нас происходит evaluation модели. В нашем случае, это примерно через каждые 30 процентов работы алгоритма. То есть, на 5 000 итерации из 16500 (примерно) мы запускаем evaluation. Evaluation тоже занимает некоторое время, но зато в конце мы можем посмотреть на метрики, которые у нас получились. Например, после первых 5 000 итерации мы видим, что наша модель дала абсолютно правильный ответ в 57% случаев, f-мера составила 69.16 (примерно). В принципе, довольно неплохо. Но, тем не менее, давайте "дотренируем" нашу модель дальше (точнее — посмотрим, как она тренировалась). 

In [ ]:
train(args, train_dataset, model, tokenizer)

Сохраняем веса дообученной модели на диск, чтобы в следующий раз не обучать модель заново.

И после того как наша модель обучилась мы можем сохранить веса нашей модели (например, в файл "bert_squad[1]_final_5epochs"). А потом, если захотим, мы сможем подгрузить веса нашей модели с помощью вот этой строчки кода, которую мы уже видели чуть ранее. [1] https://rajpurkar.github.io/SQuAD-explorer/

In [ ]:
torch.save(model.state_dict(), 'models/bert_squad_final_5epoch.pt')

Подгрузить веса модели можно так:

In [ ]:
model.load_state_dict(torch.load('models/bert_squad_5epochs.pt'))

Сопоставьте названия и определения метрик, которые измерялись в семинаре для оценки качества работы модели.  
- HasAns_exact - Процент полностью совпавших с ground truth ответов на вопросы, на которые в датасете есть ответ  
- NoAns_exact - Процент полностью совпавших с ground truth ответов на вопросы, на которые в датасете нет ответов  
- f1 - F1-мера (среднее геометрическое между precision и recall), измеряющая среднее перекрытие между предсказанным вариантом ответа и ground truth ответом  

### Оценка качества работы модели

И хотя, в процессе обучения, после каждых 5 000 итераций, модель производила evaluation на dev-датасете и выводила нам метрики, давайте всё-таки сделаем evaluation ещё раз. Чтобы не просматривать слишком много вопросов вручную, давайте отделим маленький кусочек, состоящий из всего 208 вопросов, и оценим качество работы модели на нём. Также наши метрики будут считаться быстрее на этом маленьком кусочке, чем на всём dev-датасете. Отделим наш маленький датасет с помощью следующего кода, считываем наш dev.json файл, отделяем 208 вопросов и записываем его в файл small_dev.json. 

In [ ]:
PATH_TO_DEV_SQUAD = 'dev-v2.0.json'
PATH_TO_SMALL_DEV_SQUAD = 'small_dev-v2.0.json'

with open(PATH_TO_DEV_SQUAD, 'r') as iofile:
    full_sample = json.load(iofile)
    
small_sample = {
    'version': full_sample['version'],
    'data': full_sample['data'][:1]
}

with open(PATH_TO_SMALL_DEV_SQUAD, 'w') as iofile:
    json.dump(small_sample, iofile)

Теперь объявим несколько констант, которые нам будут нужны для evaluation. Определяем максимальную длину параграфа (384 — так же как и для обучения), также определяем максимальную длину вопроса, максимальную длину ответа. 

In [ ]:
max_seq_length = 384
outside_pos = max_seq_length + 10
doc_stride = 128
max_query_length = 64
max_answer_length = 30

А теперь, с помощью функции "read_squad[1]_examples()", мы загрузим наши 208 вопросов и будем дальше с ними работать (загружаем наши вопросы с помощью вот этого кода). Здесь мы выставляем "is_training = False", потому что мы собираемся только оценить качество работы нашей модели и ничего обучать мы здесь не будем. И дальше, с помощью функции "convert_examples_to_features()" превращаем загруженные данные в фичи. Здесь мы ставим параметр "is_training" в значении False, так же как и для функции read_examples, и используем тот же самый токенайзер, что и при обучении (вот этот токенайзер). Также передаём наши примеры, загруженные с помощью "read_squad[1]_examples()". После этого вытаскиваем из наших фичей "input_ids", "input_mask", "segment_ids", "cls_index" и "p_mask". Формат, на самом деле, очень похож на тот, что мы обсуждали в прошлом семинаре, когда работали с BERT для классификации предложений. "input_ids" — это просто последовательность чисел, отождествляющих каждый токен с его номером в словаре (также, как и в прошлом семинаре). "input_mask" — это последовательность из нулей и единиц, где единицы обозначают токены предложения, а нули — паддинг. Похоже на "attention_mask" из ноутбука про классификацию с помощью BERT на прошлом семинаре. Переменная "p_mask" означает следующее — здесь мы будем маскировать единицами токены, которых не может быть в ответе, а нулями — все токены, которые могут встретиться в нашем ответе. Ну, и "cls_index" — это индекс нашего классификационного токена. 

In [ ]:
examples = read_squad_examples(
    input_file=PATH_TO_SMALL_DEV_SQUAD,
    is_training=False,
    version_2_with_negative=True)

features = convert_examples_to_features(
    examples=examples,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    doc_stride=doc_stride,
    max_query_length=max_query_length,
    is_training=False,
    cls_token_segment_id=0,
    pad_token_segment_id=0,
    cls_token_at_end=False
)

input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)

example_index = torch.arange(input_ids.size(0), dtype=torch.long)
dataset = TensorDataset(input_ids, input_mask, segment_ids, example_index, cls_index, p_mask)

Дальше мы передаём все эти переменные в тензор Dataset (вот здесь) и, дальше, создаём из него DataLoader (вот таким образом). Здесь мы передаём sampler и определяем размер батча. В нашем случае, размер батча будет равен 8. [1] https://rajpurkar.github.io/SQuAD-explorer/

In [ ]:
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=8)

Теперь давайте запустим evaluation модели на нашем маленьком кусочке dev-датасета. Evaluation может занять достаточно большое время. В нашем случае это будет не больше пары минут, и, пока, давайте рассмотрим код, который у нас есть для evaluation модели. Здесь мы используем наш "eval_dataloader", мы разбиваем его на батчи. В цикле проходимся по нашим батчам (размер каждого батча равен 8). Дальше задаём, что мы не хотим считать градиент на каждом шаге, поскольку у нас происходит процесс evaluation, а не обучения модели. Мы распаковываем наш батч и передаём полученные данные в нашу модель, чтобы получить предсказание модели. И дальше складываем полученные результаты в list под названием "all_results". Давайте посмотрим на то, как выглядит наш лист all_results.  

In [ ]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

all_results = []
for idx, batch in enumerate(tqdm.tqdm_notebook(eval_dataloader, desc="Evaluating")):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }
        inputs['token_type_ids'] = batch[2]
        example_indices = batch[3]
        outputs = model(**inputs)

    for i, example_index in enumerate(example_indices):
        eval_feature = features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        result = RawResult(unique_id    = unique_id,
                           start_logits = to_list(outputs[0][i]),
                           end_logits   = to_list(outputs[1][i]))
        all_results.append(result)

Отлично, эта ячейка кода выполнилась — смотрим на то что у нас содержится в листе all_results. На самом деле, выход не очень интерпретируемый. У нас есть некоторые айдишники, у нас есть start_logits, есть end_logits, и в целом, по вот такому вот списку достаточно сложно что-то понять. 

In [ ]:
all_results[0]

Поэтому давайте, с помощью функции "write_predictions()", которая уже есть в готовом виде в библиотеке pytorch-transformers, сформируем человеко-читаемый ответ. Для начала, нам нужно задать несколько констант. Давайте зададим параметры "n_best_size" равным 5 — это значит, что на каждый вопрос мы будем генерировать по 5 самых лучших ответов. Также нам нужно задать пути до файла с нашими предсказаниями, в файл под названием "output_1_best_file" мы будем писать айдишник нашего вопроса и один наиболее вероятный ответ на этот вопрос. Этот файл нам нужен для того, чтобы посчитать метрики, потому что для подсчёта метрик нам нужно не 5 возможных вариантов ответа, а всего один наиболее вероятный. Для того чтобы просмотреть глазами наши варианты ответа, мы всё-таки хотим видеть не один вариант, в котором наиболее уверена наша модель, а несколько вариантов. Поэтому будем записывать N лучших вариантов (в нашем случае, 5 лучших вариантов) в файл "output_n_best_file". И, кроме того, давайте поставим, что параметр "version_2_with_negative" у нас будет равен True. 

In [ ]:
n_best_size = 5
do_lower_case = True
output_prediction_file = 'output_1best_file'
output_nbest_file = 'output_nbest_file'
output_na_prob_file = 'output_na_prob_file'
verbose_logging = True
version_2_with_negative = True
null_score_diff_threshold = 0.0

Генерируем предсказания с помощью функции "write_predictions()". Файл с предсказанием будет выглядеть следующим образом. У нас есть ordered dict, в котором содержится ID вопроса и наиболее вероятный вариант ответа на него. И, как раз, вся эта информация лежит в файле "output_1_best_file". 

In [ ]:
# Генерируем файл с n лучшими ответами `output_nbest_file`
write_predictions(examples, features, all_results, n_best_size,
                    max_answer_length, do_lower_case, output_prediction_file,
                    output_nbest_file, output_na_prob_file, verbose_logging,
                    version_2_with_negative, null_score_diff_threshold)

Теперь давайте посчитаем метрики на нашем датасете. Мы будем использовать "eval_opts" — по структуре оно очень похоже на класс "train_opts", который мы использовали, только поля чуть-чуть другие. И, кроме того, с помощью функции и "evaluate_on_squad[1]()" мы можем посчитать метрики, используя наши evaluate options. Давайте посчитаем их! Получается, что абсолютно правильный ответ, с точностью до символа, был получен в 78% случаев — это достаточно неплохо. f-мера 68, а если говорить про вопросы, на которые нет ответа, то BERT смог выдать правильный ответ, то есть угадать, что на этот вопрос нет ответа, в 54% случаев. [1] https://rajpurkar.github.io/SQuAD-explorer/

In [ ]:
# Считаем метрики используя официальный SQuAD script
evaluate_options = EVAL_OPTS(data_file=PATH_TO_SMALL_DEV_SQUAD,
                             pred_file=output_prediction_file,
                             na_prob_file=output_na_prob_file)
results = evaluate_on_squad(evaluate_options)

Посмотрим глазами на вопросы и предсказанные БЕРТом ответы:

Мы смогли получить достаточно высокие значения метрик, но всё-таки давайте посмотрим глазами на вопросы и предсказанные BERT ответы, а также на каноничные ответы из нашего датасета. Давайте прочитаем файл с названием "output_n_best_file". Дальше сформируем словарь, в котором хранятся ID вопроса, а также текст вопроса, варианты ответа из нашего датасета и параграф, на котором основывался вопрос. 

In [ ]:
with open('output_nbest_file', 'r') as iofile:
    predicted_answers = json.load(iofile)
questions = {}

for paragraph in small_sample['data'][0]['paragraphs']:
    for question in paragraph['qas']:
        questions[question['id']] = {
            'question': question['question'],
            'answers': question['answers'],
            'paragraph': paragraph['context']
        }

И распечатаем вопросы-ответы BERT и ответы из нашего датасета. Для простоты понимания, давайте параграфы пока печатать не будем. Как вы видите, BERT с достаточно хорошей уверенностью (со стопроцентной уверенностью) отвечает правильно на первый вопрос. Похожая история происходит со вторым вопросом, BERT отвечает правильно с точностью до артикля, при этом, уверенность 71%. На третий вопрос мы также получаем правильный ответ с уверенностью 100%, и похожая ситуация происходит с остальными вопросами. Если говорить про вопросы, на которые в датасете ответа нет, то BERT достаточно неплохо предсказывает отсутствие ответа на вопрос, то есть BERT, примерно в 50% случаев, научился понимать, что информации, содержащаяся в параграфе, недостаточно для ответа на этот вопрос. Всего здесь 208 вопросов — вы можете более подробно изучить ответы BERT на них.

In [ ]:
for q_num, (key, data) in enumerate(predicted_answers.items()):
    gt = '' if len(questions[key]['answers']) == 0 else questions[key]['answers'][0]['text']
    print('Вопрос {0}:'.format(q_num+1), questions[key]['question'])
    print('-----------------------------------')
    print('Ground truth:', gt)
    print('-----------------------------------')   
    print('Ответы, предсказанные БЕРТом:')
    preds = ['{0}) '.format(ans_num + 1) + answer['text'] + \
             ' (уверенность {0:.2f}%)'.format(answer['probability']*100) \
             for ans_num, answer in enumerate(data)]
    print('\n'.join(preds))
#     print('-----------------------------------')   
#     print('Параграф:', questions[key]['paragraph'])
    print('\n\n')

Для того чтоб получить такие достаточно хорошие результаты, мы тренировали BERT в целых пять эпох. Что же будет, если мы потренируем BERT чуть меньшее количество времени? Как вы помните, тренировать 5 эпох BERT заняло достаточно много времени. Давайте попробуем потренировать всего одну эпоху и посмотрим, что произойдет, насколько метрики станут ниже. Мы не будем тренировать нашу модель прямо сейчас, а просто подгрузим веса модели после тренировки одной эпохи (файлик называется "bert_squad[1]_1epoch"[1] https://rajpurkar.github.io/SQuAD-explorer/). 


Подгружаем веса и ещё раз делаем evaluation. Посмотрим на метрики. Как вы видите, метрики стали чуть ниже, но не сильно ниже. На вопросы с ответом модель даёт правильный ответ в 76% случаев, сравнивая с 78% после 5 эпох, а f-мера стала равна 71. Кроме того, на вопросы без ответа — на вопросы, на которые нельзя ответить, учитывая данный параграф текста, модель научилась отвечать (точнее... не отвечать) даже лучше: 61% случаев, сравнивая с 54% после пяти эпох. Получается, что если вам нужен сравнительно быстрый результат, можно немного пожертвовать качеством работы модели и обучать модель не 7 часов, а примерно полтора часа.

На этом семинаре мы разобрали, как работает скрипт по дообучению модели BertForQuestionAnswering из библиотеки pytorch-transformers. Получить аналогичные результаты, на самом деле, вы могли бы просто склонировав репозитории и запустив скрипт "run_squad.py"[1] (например, вот так — вот таким образом). Однако, если вам вдруг понадобится что-то поменять в скрипте, модифицировать процесс обучения или даже просто обучить модель на другом, чуть менее популярном датасете, вам придётся закапываться в код библиотеки гораздо глубже, чем простой вызов готового скрипта. Этот семинар может послужить отправной точкой в более подробном освоении кода библиотеки и запуске BERT, XL трансформера, GPT-2 или любых других моделей для решения ваших задач. Успехов!
[1] https://github.com/huggingface/transformers/blob/master/examples/run_squad.py

In [ ]:
!export SQUAD_DIR=/path/to/SQUAD

python run_squad.py \
  --model_type bert \
  --model_name_or_path bert-base-cased \
  --do_train \
  --do_eval \
  --do_lower_case \
  --train_file $SQUAD_DIR/train-v1.1.json \
  --predict_file $SQUAD_DIR/dev-v1.1.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir /tmp/debug_squad/